# Assignment 2 (i21-0328 Sarim Aeyzaz) #

In [ ]:
def checkSubjects():
    '''Ask the user for number of subjects, if an exact match is found, append it into a new dataframe 1 by 1 and display'''

    # Take input of total subjects and add them into a list & create a new empty dataframe to store classes found

    total = int(input("Please enter the number of subjects to search: "))
    subjects = []
    for i in range(total):
        subjects.append(input(f"Please enter your subject {i + 1}: "))

    classes = pd.DataFrame(columns=['Subject', 'Day', 'Room', 'Time'])

    # Loop every day in timetable (note: not classDict nor labDict), every time in columns and every subject
    # If an exact match is found, get the room numbers
    # If a string is a special string, get the updated time and subject and append it into the empty dataframe
    # Else just append the dataframe with subject, day, room and timeslot and display

    # Remember, you do not loop classDict and labDict because there are certain columns you just delete there to
    # make generalized dataframes

    for day in timetable.keys():
        for timeslot in timetable[day].columns:
            for subject in subjects:
                if (timetable[day][timeslot].eq(subject).any()):
                    room = timetable[day][timetable[day][timeslot] == subject].index.values
                    if ("   " in subject):
                        newTime = subject.split("   ")[1]
                        newSubject = subject.split("   ")[0]
                        classes.loc[len(classes.index)] = [newSubject, day, room[0], newTime]
                    else:
                        classes.loc[len(classes.index)] = [subject, day, room[0], timeslot]
    display(classes.set_index('Subject'))
    return

In [ ]:
def whichEmpty():
    '''Check which values in each day are None, then get their corresponding day, time and room and
    append it into a dataframe'''

    # Create a new Dataframe to store empty classes
    # Loop through every day(dataframe) and timeslot(column) and see if any classes labelled as "None" appear
    # If they do, get the room numbers
    # Loop through rooms and start adding them with their day and timeslot into the empty classes dataframe and display

    emptyClasses = pd.DataFrame(columns=['Day', 'Room', 'Time'])

    for day in classDict.keys():
        for timeslot in classDict[day].columns:
            if (classDict[day][timeslot].eq("None").any()):
                rooms = classDict[day][classDict[day][timeslot] == 'None'].index.values
                for room in rooms:
                    emptyClasses.loc[len(emptyClasses.index)] = [day, room, timeslot]

    display(emptyClasses)
    return

In [ ]:
def leastBusyClass():
    '''Cycle through all rows in all days and calculate how many classes were held each day for each room,
    then append (and sum depending on the condition) into a dictionary and display the least busy class'''

    # Create an empty dictionary, for each day(dataframe)'s row(Room Numbers), check if the class is in the dictionary
    # If it isn't, add it alone with the number of times 'None' appeared in the row
    # If it is, add the current value with the number of times 'None appeared in the row'
    # You do 8 - because you're trying to find how many classes there were instead of how many empty spaces there were

    busiestClass = {}

    for day in classDict.keys():
        for index, row in classDict[day].iterrows():
            if (index in busiestClass):
                busiestClass[index] = busiestClass[index] + (8 - sum(row[0:].isin(['None'])))
            else:
                busiestClass[index] = (8 - sum(row[0:].isin(['None'])))

    # Get minimum value and print out the classes which match the minimum total classes held

    minValue = min(busiestClass.values())
    print("The least busiest classes are: ")
    for index, value in busiestClass.items():
        if (busiestClass[index] == minValue):
            print(f"{index} ")

In [ ]:
def busyLab():
    '''Cycle through all rows in all days and calculate how many labs were held each day for each room,
    then append (and sum depending on the condition) into a dictionary and display the busiest lab'''

    # Create an empty dictionary, for each day(dataframe)'s row(Room Numbers), check if the class is in the dictionary
    # If it isn't, add it alone with the number of times 'None' appeared in the row
    # If it is, add the current value with the number of times 'None appeared in the row'
    # You do 8 - because you're trying to find how many labs there were instead of how many empty spaces there were

    busiestLab = {}
    labDict['Friday']["Something"] = "None" # 'Equalising' the friday sheet since other sheets have 1 extra "None" column

    for day in labDict.keys():
        for index, row in labDict[day].iterrows():
            if (index in busiestLab):
                busiestLab[index] = busiestLab[index] + (4 - sum(row[0:].isin(['None'])))
            else:
                busiestLab[index] = (4 - sum(row[0:].isin(['None'])))

    # Get maximum value and print out the labs which match the maximum total classes held

    maxValue = max(busiestLab.values())
    print("The busiest labs are: ")
    for index, value in busiestLab.items():
        if (busiestLab[index] == maxValue):
            print(f"{index} ")

In [ ]:
import pandas as pd
import numpy as np
from IPython.display import display, clear_output

# Read from excel file, keep ALL of the sheets, set each dataframe's header column starting from row 4 and set indexes as Room

df = pd.read_excel("TimeTable, FSC, Fall-2022.xlsx", sheet_name = None, header = 4, index_col = 'Room')
df.pop('Instructions') # Deleting this, it's useless
timetable = df         # Copy over the dictionary

for day in df.keys(): # Fill each NaN value in dataframes with "None"
    timetable[day] = df[day].fillna("None")

# Create a Lab classes sub-frame which has proper column and drop any column being Unnamed
# This is useful later when calculating busiest lab

labDict = { 'Monday':df['Monday'].copy(), 'Tuesday':df['Tuesday'].copy(), 'Wednesday':df['Wednesday'].copy(), 'Thursday':df['Thursday'].copy(), 'Friday':df['Friday'].copy()}
for day in labDict.keys():
    labDict[day] = labDict[day].loc['Lab':]
    labDict[day].columns = labDict[day].loc['Lab']
    labDict[day] = labDict[day][1:]
    labDict[day] = labDict[day].drop(labDict[day].columns[labDict[day].columns.str.contains('None')], axis = 1)
    # display(labDict[day])

# Create a classes sub-frame which has classrooms only and drop any column being Unnamed
# This is useful later when calculating least busiest class

classDict = { 'Monday':df['Monday'].copy(), 'Tuesday':df['Tuesday'].copy(), 'Wednesday':df['Wednesday'].copy(), 'Thursday':df['Thursday'].copy(), 'Friday':df['Friday'].copy()}
for day in classDict.keys():
    if (day == 'Friday'):
        classDict[day] = classDict[day].loc[:'A-108']
    else:
        classDict[day] = classDict[day].loc[:'A-211']
    classDict[day] = classDict[day].drop(classDict[day].columns[classDict[day].columns.str.contains('Unnamed')], axis = 1)
    # display(classDict[day])

# MAIN MENU Starts here, repeat until -1

check = 0
while (check != "-1"):
    check = input("Welcome to the Menu!\nPlease select an option\n\t1 - Check subjects\n\t2 - Return list of empty classes\n\t3 - Return the least busiest class of all week\n\t4 - Return the busiest labs of the week\n\t-1 - Exit\n\n")
    match check:
        case "1":
            clear_output()
            checkSubjects()
        case "2":
            clear_output()
            whichEmpty()
        case "3":
            clear_output()
            leastBusyClass()
        case "4":
            clear_output()
            busyLab()
        case default:
            clear_output()
            print("Incorrect Choice")
clear_output()